# Test Audio Sender

This file is used for sending audio data to the device. It measures the accuracy based on individual audio files.

**Note that the software on the MCU must match this!**

In [219]:
AUDIO_FOLDER_PATH = "./test"
POSITIVE_PATH = f"{AUDIO_FOLDER_PATH}/positive"
NEGATIVE_PATH = f"{AUDIO_FOLDER_PATH}/negative"

In [220]:
import numpy as np
import pandas as pd
import gc
import os
import librosa
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import serial
import struct

In [221]:
# Collecte positive audio files and their classes
positive_audio_files = []
positive_audio_file_classes = []
# Look each file in the audio folder
audio_class_folders = os.listdir(POSITIVE_PATH)
# Loop each class folder
for audio_class_folder in audio_class_folders:
    # Assemble full audio class folder path.
    audio_class_folder_path = os.path.join(POSITIVE_PATH, audio_class_folder)
    print("Processing class folder: ", audio_class_folder_path)
    # Get all files in the audio class folder
    audio_class_files = os.listdir(audio_class_folder_path)
    # Loop each audio file in the audio class folder
    for audio_class_file in audio_class_files:
        # Assemble full audio file path.
        audio_file_path = os.path.join(audio_class_folder_path, audio_class_file)
        # Append the audio file path to the positive_audio_files list
        positive_audio_files.append(audio_file_path)
        # Append the audio class to the audio_file_classes list
        positive_audio_file_classes.append(audio_class_folder)
print("positive_audio_files length: ", len(positive_audio_files))
print("positive_audio_file_classes length: ", len(positive_audio_file_classes))

Processing class folder:  ./test/positive/comm
positive_audio_files length:  1
positive_audio_file_classes length:  1


In [222]:
positive_audio_file_classes[0]

'comm'

In [223]:
# Collecte negative audio files and their classes
negative_audio_files = []
negative_audio_file_classes = []
# Look each file in the audio folder
audio_class_folders = os.listdir(NEGATIVE_PATH)
# Loop each class folder
for audio_class_folder in audio_class_folders:
    # Assemble full audio class folder path.
    audio_class_folder_path = os.path.join(NEGATIVE_PATH, audio_class_folder)
    print("Processing class folder: ", audio_class_folder_path)
    # Get all files in the audio class folder
    audio_class_files = os.listdir(audio_class_folder_path)
    # Loop each audio file in the audio class folder
    for audio_class_file in audio_class_files:
        # Assemble full audio file path.
        audio_file_path = os.path.join(audio_class_folder_path, audio_class_file)
        # Append the audio file path to the negative_audio_files list
        negative_audio_files.append(audio_file_path)
        # Append the audio class to the audio_file_classes list
        negative_audio_file_classes.append(audio_class_folder)
print("negative_audio_files length: ", len(negative_audio_files[0:10]))
print("negative_audio_file_classes length: ", len(negative_audio_file_classes[0:10]))

negative_audio_files length:  0
negative_audio_file_classes length:  0


In [224]:
# Shuffle the audio files and classes with the same seed.
seed = 42
np.random.seed(seed)
np.random.shuffle(positive_audio_files)

np.random.seed(seed)
np.random.shuffle(positive_audio_file_classes)

np.random.seed(seed)
np.random.shuffle(negative_audio_files)

In [225]:
# Hot end code the labels.
label_encoder = LabelEncoder()
positive_audio_file_classes_categorical = to_categorical(label_encoder.fit_transform(positive_audio_file_classes))
print("Example of audio_file_classes_categorial: ", positive_audio_file_classes_categorical[0:10])

Example of audio_file_classes_categorial:  [[1.]]


In [226]:
# Connect to MCU via serial.
print("Configuring serial port...")
ser = serial.Serial(
    port='/dev/ttyACM0',  # Change this to your actual port, e.g., 'COM3' on Windows, '/dev/ttyS0' on Linux
    baudrate=921600,       # Set baud rate to 921600
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    timeout=0.1,           # Set timeout for reading
    write_timeout=None     # Wait indefinitely until all data is sent
)

Configuring serial port...


In [227]:
if ser.is_open:
    print(f"Serial port {ser.port} opened at {ser.baudrate} baud.")

Serial port /dev/ttyACM0 opened at 921600 baud.


In [228]:
def calc_accuracy(y_true, y_pred):
    success = 0
    for i in range(len(y_pred)):
        if y_pred[i] == y_true[i]:
            success += 1
    res = round(success / len(y_pred), 3)
    return res

In [229]:
audioSent = 0
audioFilesProcessed = 0
results = []
realResults = []

In [230]:
positive_audio_file_classes_categorical[0]

array([1.])

In [231]:
def streamAudioFile(file_path, realClass, i):
    global audioSent
    global results
    global realResults
    global audioBuffer
    voted = False
    # Append the real result to the results list.
    realResults.append(np.argmax(realClass))
    #print(f"Adding realResults: {realResults}")
    # Check file format.
    if not file_path.endswith('.wav'):
        print("Error: Only .wav files are supported.")
        return
    # Load the audio file.
    audio_data, sr = librosa.load(file_path, sr=None)
    # Convert sample rate to 16kHz.
    if sr != 16000:
        audio_data = librosa.resample(y=audio_data, orig_sr=sr, target_sr=16000)
        sr = 16000
    # Convert the audio data to a numpy array.
    audio_data = np.array(audio_data, dtype=np.float32)
    # Send the audio data to the MCU.
    for j in range(len(audio_data)):
        data_to_send = audio_data[j].tobytes()
        ser.write(data_to_send)

        while ser.in_waiting > 0:
            response = ser.readline().decode('utf-8').strip()
            if response.startswith("e:"):
                print(response)
                return
            if response.startswith("c:"):
                print(response)
            if response.startswith("s:"):
                print(response)
            if response.startswith("v:"):
                voted = True
                print(response)
                #print(f"Adding {int(response[2:])} to results")
                response = int(response[2:])
                results.append(response)
                if len(results) > len(realResults):
                    realResults.append(9)
                print(f"Accuracy: {calc_accuracy(realResults, results)}")
    # If result is shorter than realResults, append 9 to results.
    if i > 0 and len(results) < len(realResults) and not voted:
        results.append(9)
        print(f"Accuracy: {calc_accuracy(realResults, results)}")

In [232]:

total_positive_audio_files = len(positive_audio_files)

# Loop each audio file.
negative_audio_file_pointer = 0
for i in range(total_positive_audio_files):
    # Get audio file path.
    audio_file = positive_audio_files[i]
    # Print audio file path and class.
    print("Processing audio file: ", audio_file)
    print("Processing audio file class: ", positive_audio_file_classes[i])
    # Stream positive audio file to MCU.
    streamAudioFile(audio_file, positive_audio_file_classes_categorical[i], i)
    # Print nagtive audio file path and class.
    print("Processing negative audio file: ", negative_audio_files[negative_audio_file_pointer])
    # Stream negative audio file to MCU.
    streamAudioFile(negative_audio_files[negative_audio_file_pointer], [0,0,0,1], i)
    # Increment the negative audio file pointer.
    negative_audio_file_pointer = (negative_audio_file_pointer + 1) % len(negative_audio_files)

Processing audio file:  ./test/positive/comm/2019-10-22-08-40_Fraunhofer-IDMT_30Kmh_178946_M_D_BL_ME_CH12.wav
Processing audio file class:  comm
s:-127 -127 -48 -42 -35 -37 -37 -39 -26 12 16 16 23 18 16 14 7 16 1 2 2 1 2 0 9 8 5 8 6 8 7 6 5 7 7 4 3 3 1 4 9 7 5 2 3 1 1 2 7 2 2 2 2 2 0 2 9 7 4 8 6 7 6 6 5 11 3 5 4 5 7 6 6 11 9 9 8 11 10 13 10 7 8 9 11 9 10 8 9 4 6 7 9 7 11 9 4 5 7 5 9 6 9 8 9 9 8 5 6 5 9 7 7 7 9 9 6 7 9 8 8 10 8 10 5 10 9 7
c: [-2592,-2792,-3963,-7547] voted for: 3
s:-40 -39 -44 -43 -39 -41 -37 -39 15 19 14 15 19 18 16 17 -1 1 0 -1 1 1 4 4 8 6 6 7 6 4 10 8 4 1 3 5 1 1 0 3 5 1 3 5 3 5 3 7 2 2 6 6 5 4 7 6 4 4 5 7 4 4 6 5 8 4 7 3 5 1 6 8 12 11 13 9 8 8 10 9 7 7 7 7 6 4 5 5 12 8 7 10 9 10 8 9 10 8 6 8 6 8 7 7 6 6 5 6 5 5 6 5 4 6 7 7 6 7 8 6 7 7 9 6 8 6 7 4
c: [-2675,-2783,-4032,-7660] voted for: 3
s:-45 -47 -42 -45 -43 -43 -39 -41 15 15 14 13 13 13 17 17 4 2 2 5 3 1 7 4 8 3 5 6 3 3 5 4 -1 1 1 3 1 0 4 2 0 5 6 8 8 5 5 5 2 5 5 5 4 6 2 3 5 6 8 6 8 6 8 6 5 10 9 6 7 6 9 5 13 14 9 

IndexError: list index out of range

In [ ]:
# Close the serial port
ser.close()